In [1]:
from errorprop import *

# Introduction

In this jupyter notebook we'll look at how to work with `Quantity`, `MeasuredQuantity` and `DerivedQuantity` which are the basic building blocks. We'll see how to translate lab data into python code and how to do maths with automatic error propagation. Finally, we'll see how to handle many quantities at once and how to take weighted averages.

# What is a Quantity?

Measurements from the lab are represented by three numbers:
1. $x$ value of measurement
2. $\Delta x_{sys}$ systematic error (precision)
3. $\Delta x_{stat}$ statistical error (accuracy)

This set of three numbers we call a `Quantity`.

**Example:** We measure the period of an oscillation three times and get (2.77, 2.81, 2.93) seconds which is on average 2.83s. The clock used for the measurement has a precision of 1 second, so we estimate the systematic error as 0.5s. The statistical error corresponds to $\frac{2 \sigma}{\sqrt{n}} = 0.08$s with $\sigma$ the standard deviation and $n=3$ the number of measurements (see [IF0](https://www.ap.physik.unibas.ch/PDF/Manuals/German/IF0.pdf) "statistischer Fehler des Mittelwerts"). Hence the quantity $T$ (period) which we have measured in the lab is written as follows:

$$
T = (2.83 \pm 0.50 \pm 0.08)s
$$

To represent this in code we use the `Quantity` object:

In [2]:
T = Quantity(value=2.83, err_sys=0.5, err_stat=0.08)
print(T)

2.83 ± 0.5 ± 0.08


`Quantity` is the basic building block. Everything is a quantity – but there are "sub-categories" such as the `MeasuredQuantity` and the `DerivedQuantity`. In the following we'll have a closer look at these and how to work with them. We'll see how do maths with quantities and how to take care of error propagation.

## Measured Quantity

The measured quantity is received from the lab by measuring multiple times. Let's say you measure the period $T$ of a pendulum multiple times and let's say you have estimated the systematic error as $\Delta T_{sys} = 0.3$s

This would translate into code as follows:

In [3]:
measurements = np.array([2.77,2.81,2.93,2.95,2.49,2.81,2.95,2.76])
T = MeasuredQuantity(measurements, err_sys=0.3)
print(T)
print('Data\t', T.measurements)
print('Value\t', T.value)
print('Err Stat', T.err_stat)
print('Err Sys\t', T.err_sys)

2.809 ± 0.3 ± 0.1
Data	 [2.77 2.81 2.93 2.95 2.49 2.81 2.95 2.76]
Value	 2.8087500000000003
Err Stat 0.10002734001261855
Err Sys	 0.3


The `MeasuredQuantity` receives a list of measurements and calculates mean and stat error automatically. You can access individual information by the following properties
* `T.measurements` (underlying data)
* `T.value` (mean value)
* `T.err_stat` and `T.err_sys` (errors)

By default, the 2-sigma confidence interval is taken as statistical error. Recall from [IF0](https://www.ap.physik.unibas.ch/PDF/Manuals/German/IF0.pdf):
* $1\sigma$ confidence interval includes 68.6% of all measurements
* $2\sigma$ confidence interval includes 95.4% of all measurements
* $3\sigma$ confidence interval includes 99.7% of all measurements

You can change this with the `sigma_interval` argument:

In [4]:
T = MeasuredQuantity(measurements, err_sys=0.3, sigma_interval=3)
print(T)

2.809 ± 0.3 ± 0.15


Sometimes you know the value and errors of a quantity right away without having to take the mean over many measurements. This is the case for example if you measure something only once. For this you use the `Quantity` object.

Let's say you have measured the length $l$ of the pendulum to be 1.84m with a systematic error $\Delta l_{sys} = 0.7$cm and no statistical error.

This translates into code as follows:

In [5]:
l = Quantity(1.84, err_sys=0.7E-2, err_stat=0)
print(l)

1.84 ± 0.007


**Note**: `0.7E-2` in python means $0.7 \cdot 10^{-2}$.

**Note**: `MeasuredQuantity` is a type of `Quantity`, moreover `Quantity` is the general case and `MeasuredQuantity` is a special case. The `MeasuredQuantity` simply does the calculation of mean and statistical error automatically.

## Derived Quantity

The derived quantity is a function of other quantities: $Q = f(q_1, ..., q_N)$. The statistical and systematic errors of $Q$ result from the errors of the underlying $q_i$ and derivatives of $f$ with respect to $q_i$. E.g. systematic error ([IF0](https://www.ap.physik.unibas.ch/PDF/Manuals/German/IF0.pdf)):

$$
\Delta Q_{sys} = \left| \frac{\partial f}{\partial q_1} \Delta q_1  + ... + \frac{\partial f}{\partial q_n} \Delta q_n \right|
$$

**Example**: We want to calculate the gravitational acceleration $g$ based on the period and length measurements for which we use the following equation:

$$
g = \frac{4\pi^2 l}{T^2}
$$

In that case $g$ is a derived quantity, i.e. a function of $l$ and $T$ which themselves are quantities with errors.

For error calculation we need the following derivatives:

$$
\frac{\partial g}{\partial l} = \frac{4 \pi^2}{T^2} \quad \quad
\frac{\partial g}{\partial T} = -\frac{8 \pi^2 l}{T^3}
$$

This translates to code using the `DerivedQuantity` object:

In [6]:
g = lambda l, T: 4*np.pi**2*l/T**2
# partial derivatives
g_l = lambda l, T: 4*np.pi**2/T**2
g_T = lambda l, T: -8*np.pi**2*l/T**3
G = DerivedQuantity(value_function=g, partial_derivatives=[g_l, g_T], args=[l, T])
print(G)

9.208 ± 2.002 ± 0.984


**Note**: `np.pi` represents $\pi$ in python (numpy) and `a**b` is the general notation for $a^b$ in python.

**Note**: `g = lambda l, T: 4*np.pi**2*l/T**2` is short for
```python
def g(l, T):
  return 4*np.pi**2*l/T**2
```

The `DerivedQuantity` takes care to do error propagation for statistical and systematic errors by using the provided partial derivatives and looking at the errors of the quantities passed as arguments.

# Basic Arithmetics with Quantities

Previously we have looked at how to calculate the value of the derived quantity $g$ (gravitational acceleration) from other quantities $l$ (pendulum length) and $T$ (period). By providing the partial derivatives we were able to get the corresponding error propagations for $g$.

**Motivation**: Can we do error propagation automatic without providing partial derivatives?

**Idea**: Solve error propagation for basic arithmetics (addition, subtraction, etc.) Most function will just be combinations of these.

Consider the function for addition $a(x,y) = x+y$ it has partial derivatives $a_x = a_y = 1$. Given $x$ and $y$ are quantities we have the following systematic error using error propagation

$$
\Delta a_{sys} = \left| 1 \cdot \Delta x_{sys} \right| + \left| 1 \cdot \Delta y_{sys} \right|
$$

We can translate this into code as a derived quantity (defining quantities $x$ and $y$ with random values)

In [7]:
x = Quantity(1,err_sys=0.5,err_stat=0.7)
y = Quantity(4,err_sys=0.3,err_stat=0.9)
a = lambda x, y: x+y
# partial derivatives
a_x = lambda x, y: 1
a_y = lambda x, y: 1
# here A represents simply the addition x+y including error propagation from x and y
A = DerivedQuantity(value_function=a, partial_derivatives=[a_x, a_y], args=[x, y])
print(A)

5 ± 0.8 ± 1.14


So whenever we add two quantities we know how to do error propagation because the function (addition) is known, as well as its derivatives. What about automating this process? Using python operator overloading we can define the addition operation on the `Quantity` object and we can configure it in such a way, that it returns a `DerivedQuantity` just as above.

What we want to achieve is the following:
```
x = Quantity(...) or MeasuredQuantity(...) or DerivedQuantity(...)
y = Quantity(...) or MeasuredQuantity(...) or DerivedQuantity(...)
z = x+y = DerivedQuantity (with automatic error propagation)
```

And then we do the same for subtraction, multiplication, division and exponential ($a^b$). If we provide the code to create a DerivedQuantity for all of the basic arithmetic operations we are able to do basic maths on quantities while keeping track of error propagation.

This has been done in the `Quantity` python class (see code for details). Now we can simplify the calculation of gravitational acceleration as follows:

In [8]:
G = 4*np.pi**2*l/T**2
print(G)

9.208 ± 2.002 ± 0.984


Note that we get the same result as with the `DerivedQuantity` approach.

# Dealing with a list of quantities

Sometimes you need to work with a list of quantities. This is most certainly the case when you take measurements of a varying parameter. Let's say you make voltage measurements and increase the voltage each time. So you don't want to take the average over these voltages and represent it as one quantity (as you would do using `MeasuredQuantity`), rather each measurement should represent a quantity, so you have a list of quantities.

You could do this as follows:

In [9]:
err_sys = 1
V = np.array([Quantity(292.4, err_sys), Quantity(260, err_sys), Quantity(231, err_sys)])
V

array([292.4 ± 1, 260 ± 1, 231 ± 1], dtype=object)

That's very tedious... So there's a shortcut! Use the `create_list_of_quantities` function:

In [10]:
# as you see you could also specify a statistical error
V = create_list_of_quantities(np.array([292.4, 260, 231]), err_sys=1, err_stat=0)
V

array([292.4 ± 1, 260.0 ± 1, 231.0 ± 1], dtype=object)

Previously you have specified one value for `err_sys` which is populated to alle measurements.
If you have individual errors for each measurement, you can also pass a list of errors (one for each measurement)
Let's say the multimeter has specified 2% error in the specs, you could achieve that as follows:

In [11]:
voltages = np.array([292.4, 260, 231])
V = create_list_of_quantities(voltages, err_sys=voltages*0.02)
V

array([292.4 ± 5.848, 260.0 ± 5.2, 231.0 ± 4.62], dtype=object)

You can do maths on the list of quantities as well. The operations will be "propagated" to each individual quantities.
Let's do some random math on the voltages $e^{\frac{\sqrt{V}}{2}-5}$:

In [12]:
np.exp(np.sqrt(V)/2-5)

array([34.808 ± 2.976, 21.376 ± 1.723, 13.455 ± 1.022], dtype=object)

Technically speaking `V` is a list of `Quantity` objects. In general that should not bother as you can handle them as if they were numbers. But if you ever need access to the numeric data type, you can do as follows:

In [13]:
# access underlying data
values = q_value(V)
print(values)
# compare the types...
print(type(V[0]))
print(type(values[0]))
# or if you need access to sys and stat error:
# q_sys_error(V)
# q_stat_error(V)

[292.4 260.  231. ]
<class 'errorprop.Quantity'>
<class 'numpy.float64'>


# Reading Data From Excel

Instead of copying the data into python code you can also import directly from excel using the [pandas](https://pandas.pydata.org/) library.

The following code reads the data from `sample_measurements.xlsx` into a DataFrame, which is pandas representation for a spreadsheet. The sheet contains voltage and diameter [cm] measurements.

In [14]:
import pandas as pd
df = pd.read_excel('sample_measurements.xlsx')
df.head()

,V,d
0,292.4,7.9
1,260.0,7.3
2,231.4,7.6
3,201.3,6.9
4,171.2,6.0


Let's convert the data into a list of quantities

In [15]:
volt = df['V']
# convert diameter [cm] to radius [m]
radius = df['d']/2/100
V = create_list_of_quantities(data=volt, err_sys=volt*0.01)
r = create_list_of_quantities(data=radius, err_sys=0.5E-3)
print('V =', V)
print('r = ', r)

V = [292.4 ± 2.924 260.0 ± 2.6 231.4 ± 2.314 201.3 ± 2.013 171.2 ± 1.712
 124.0 ± 1.24 96.9 ± 0.969 178.3 ± 1.783 229.8 ± 2.298 289.0 ± 2.89
 301.1 ± 3.011 280.6 ± 2.806 152.0 ± 1.52 132.2 ± 1.322 108.4 ± 1.084]
r =  [0.04 0.036 0.038 0.034 0.03 0.026 0.022 0.034 0.037 0.039 0.04 0.04 0.028
 0.025 0.024]


# Weighted Average

When you have a list of quantities each with an individual error, and you want to take the average over them, it makes sense to weigh quantities with large errors less than quantities with little error. This is achieved with the [weighted average](https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Mathematical_definition).

Let's say the voltage and radius data we imported in the previous section represents the radius of the circular trajectory of an electron as a function of applied voltage (see [IE4](https://ap.physik.unibas.ch/PDF/Manuals/English/IE4en.pdf) but meaning of the physical phenomenon is totally irrelevant here it just serves as a good example for weighted average). From each of them you can calculate the specific charge $q_m$ of the electron (which is a constant). Since the voltage error varies we want to take the weighted average over all the resulting specific charges.

The formula to get specific charge is:

$$
q_m = 2 \frac{V}{(r B)^2}
$$

And let's assume we also measured the magnetic field $B$. So let's calculate $q_m$.

In [16]:
B = Quantity(9.98E-4, err_sys=5.1E-5)
qm = 2*V/(r*B)**2
print(qm)

[376315485569.938 ± 51751230905.523 391883097528.017 ± 54707534503.7
 321784465683.319 ± 44573647993.52 339605330069.721 ± 47948848047.435
 381970799760.287 ± 55591167764.712 354567438883.429 ± 53163933219.99
 402019693837.069 ± 63382004527.175 314320496583.635 ± 44581345397.207
 337066420704.205 ± 46930241729.667 381537774189.58 ± 52593240245.553
 368612129686.75 ± 50461440160.851 361129019326.008 ± 49662774938.747
 375770585106.037 ± 55348049163.618 424737250051.204 ± 64646882046.872
 394150852974.171 ± 60997840165.239]


And now take the weighted average

In [17]:
qm_weighted = WeightedAverage(qm)
# scale down the value by 10^11 as it's huge by nature
qm_weighted*1E-11

3.622 ± 0.135

**Note**: Implementation uses the following formulas to compute the weighted average:
* Mean: $\overline x = \sum \frac{w_i \cdot x_i}{w_i}$
* Weights: $w_i = \frac{1}{\Delta x_i^2}$
* Error: $\Delta x = \sqrt{\frac{1}{w_i}}$
* $x_i :=$ quantities passed to `WeightedAverage`
* $\Delta x_i :=$ total error (stat + sys) of $x_i$